<a href="https://colab.research.google.com/github/oingoboing0/sql_helper/blob/main/SQL_helper_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --force-reinstall --upgrade --no-cache-dir "llama-cpp-python<0.3" \
&& pip install -U "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" "unsloth[gguf]"\
&& !pip install transformers==4.41.2 --upgrade\
&& !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo\
&&    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer\
&&    !pip install --no-deps unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 43.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 162.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 277.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 387.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 316.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 291.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.90-cp311-cp311-linux_x86_64.whl size=3414589 sha256=158e20c86b2142ea1cdbfadeca101427405cee4b7d155b5045cdea8f6f89ab2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fn8invac/wheels/f8/bd/3d/dcb25387572a6fe6c9c2953597c4c4d6dd985a425475da444d
Successfully built llama-cpp

In [1]:
import unsloth, llama_cpp
print("Unsloth :", unsloth.__version__)
print("llama-cpp-python:", llama_cpp.__version__)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth : 2025.6.12
llama-cpp-python: 0.2.90


In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [3]:
from unsloth import FastLanguageModel
import torch

model_name =   "unsloth/Llama-3.2-1B-bnb-4bit"#"unsloth/gemma-3-4b-it-unsloth-bnb-4bit" #

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model / tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
from unsloth import FastLanguageModel
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ['q_proj','k_proj','o_proj','gate_proj','up_proj','down_proj',],
    lora_alpha = 16, lora_dropout = 0, bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.6.12 patched 16 layers with 0 QKV layers, 16 O layers and 16 MLP layers.


In [5]:
alpaca_prompt = ("""
    Below is an instruction that describes a task, paired with an input that provides further context.
    Write a response that appropriately completes the request.\n\n

    ### Instruction:\n Company database:{} \n\n

    ### Input:\n SQL Prompt:{}\n\n

    ### Response:
    SQL:{}

    Explanation:{}
    """
)
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    company_databases = examples["sql_context"]
    prompts           = examples["sql_prompt"]
    sqls              = examples["sql"]
    explanations      = examples["sql_explanation"]

    texts = []
    for company_database, prompt, sql, explanation in zip(
        company_databases, prompts, sqls, explanations
    ):
        text = alpaca_prompt.format(company_database, prompt, sql, explanation) + EOS_TOKEN
        texts.append(text)
    return {"text": texts, }

pass
import json
import pandas as pd
from datasets import load_dataset

df = load_dataset("gretelai/synthetic_text_to_sql", split = 'train')
dataset = df.map(formatting_prompts_func , batched= True,)

README.md: 0.00B [00:00, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [6]:
## check if it worked with
#dataset['text']

In [7]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer

trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = dataset,
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    dataset_num_proc    = 2,
    packing             = False,   # Can make training ~5× faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size    = 2,
        gradient_accumulation_steps    = 4,
        warmup_steps                   = 5,
        # num_train_epochs             = 1,   # Uncomment for a full-epoch run instead of max_steps.
        max_steps                      = 60,
        learning_rate                  = 2e-4,
        fp16                           = not is_bfloat16_supported(),
        bf16                           = is_bfloat16_supported(),
        logging_steps                  = 1,
        optim                          = "adamw_8bit",
        weight_decay                   = 0.01,
        lr_scheduler_type              = "linear",
        seed                           = 3407,
        output_dir                     = "outputs",
    ),
)



Unsloth: Tokenizing ["text"]:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,616,832 of 1,246,431,232 (0.85% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: oingoboing0 (oingoboing0-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.360900
2,0.356300
3,0.292500
4,0.224200
5,0.133000
6,0.040600
7,0.027000
8,0.014600
9,0.016100
10,0.015300


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.13 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 27.91it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model.bin...


In [ ]:
!zip -r /content/unsloth_model.zip /content/model


In [ ]:
ls


In [ ]:
#from google.colab import files
#files.download('/content/unsloth_model.zip')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # follow the auth link when prompted


In [ ]:
!cp /content/unsloth_model.zip /content/drive/MyDrive/


In [ ]:
!ls /content/drive/MyDrive/


In [ ]:
!ls /content/drive/MyDrive/unsloth_model.zip


In [ ]:
!du -h /content/drive/MyDrive/unsloth_model.zip


In [ ]:
# 3. Verify it’s there
!ls -lh /content/drive/MyDrive/unsloth_model.zip
